# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [1]:
import pandas as pd
import json

from pandas import json_normalize

## imports for Python, Pandas

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [2]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [3]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

,name,population
0,Dade,12345
1,Broward,40000
2,Palm Beach,60000
3,Summit,1234
4,Cuyahoga,1337


In [4]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,state,shortname,info.governor
0,Dade,12345,Florida,FL,Rick Scott
1,Broward,40000,Florida,FL,Rick Scott
2,Palm Beach,60000,Florida,FL,Rick Scott
3,Summit,1234,Ohio,OH,John Kasich
4,Cuyahoga,1337,Ohio,OH,John Kasich


****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [6]:
# load json as string
json.load((open('data/world_bank_projects_less.json')))

In [7]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Name': 'Education for all', 'Percent': 100}","[{'name': 'Education for all', 'code': '65'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Name': 'Other economic management', 'Percent...","[{'name': 'Other economic management', 'code':...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en


****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [9]:
# Load data in JSON using json.load
world_projects_json = json.load((open('data/world_bank_projects.json')))
# world_projects_json

In [10]:
# Setup the world projects data with countryshortname as meta
world_projects = json_normalize(
    data=world_projects_json, 
    record_path=['mjtheme_namecode'], 
    meta=['countryshortname']
)

world_projects

,code,name,countryshortname
0,8,Human development,Ethiopia
1,11,,Ethiopia
2,1,Economic management,Tunisia
3,6,Social protection and risk management,Tunisia
4,5,Trade and integration,Tuvalu
...,...,...,...
1494,10,Rural development,Indonesia
1495,9,Urban development,Kenya
1496,8,Human development,Kenya
1497,5,Trade and integration,Kenya


In [11]:
# 1. Find the 10 countries with the most projects

world_projects['countryshortname'].value_counts().head(10)

Indonesia             56
India                 51
Vietnam               43
Bangladesh            41
Brazil                41
China                 40
Africa                39
Yemen, Republic of    34
Morocco               32
Mozambique            31
Name: countryshortname, dtype: int64

In [12]:
# 2. Find the top 10 major project themes (using column 'mjtheme_namecode')

world_projects[(world_projects.name.str.len() > 0)].name.value_counts().head(10)

Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Economic management                              33
Name: name, dtype: int64

In [13]:
# 3. Create a dataframe with the missing names filled in.

# Create a key-value dataframe similar to a json object to fill values
# Most of the data that is empty has a CODE that is associated with it
# Based on this code, we will fill to the value that is assigned using some type of loop/function

key_value = world_projects[(world_projects['code'].str.len() > 0) & (world_projects['name'].str.len() > 0)]
key_value = key_value[['code', 'name']].drop_duplicates().set_index('code').sort_values(by=['code'])

code_name = key_value.name.to_dict()
code_name

{'1': 'Economic management',
 '10': 'Rural development',
 '11': 'Environment and natural resources management',
 '2': 'Public sector governance',
 '3': 'Rule of law',
 '4': 'Financial and private sector development',
 '5': 'Trade and integration',
 '6': 'Social protection and risk management',
 '7': 'Social dev/gender/inclusion',
 '8': 'Human development',
 '9': 'Urban development'}

In [14]:
# Once we have this json key pair, we can iterate through the entire rows and fill the name with the corresponding code
# We will be using the .apply function

def fill_empty_values(row):
    name = row['name']
    if len(name) <= 0:
        name = code_name.get(row['code'])
    
    return name

testing = world_projects.copy()
testing.apply(fill_empty_values, axis=1)
testing

,code,name,countryshortname
0,8,Human development,Ethiopia
1,11,,Ethiopia
2,1,Economic management,Tunisia
3,6,Social protection and risk management,Tunisia
4,5,Trade and integration,Tuvalu
...,...,...,...
1494,10,Rural development,Indonesia
1495,9,Urban development,Kenya
1496,8,Human development,Kenya
1497,5,Trade and integration,Kenya
